## Check recording info

In [9]:
import numpy as np
import subjects

sess = subjects.sd.ratKday1[0]
print(sess.recinfo)
sess.recinfo.channel_groups

filename: /data/Clustering/SleepDeprivation/RatK/Day1/RatK_Day1_2019-08-06_03-44-01.xml 
# channels: 134
sampling rate: 30000
lfp Srate (downsampled): 1250


array([[13, 2, 14, 1, 15, 0, 12, 3, 11, 4, 10, 5, 9, 6, 8, 7],
       [29, 18, 30, 17, 31, 16, 28, 19, 27, 20, 26, 21, 25, 22, 24, 23],
       [45, 34, 46, 33, 47, 32, 44, 35, 43, 36, 42, 37, 41, 38, 40, 39],
       [61, 50, 62, 49, 63, 48, 60, 51, 59, 52, 58, 53, 57, 54, 56, 55],
       [77, 66, 78, 65, 79, 64, 76, 67, 75, 68, 74, 69, 73, 70, 72, 71],
       [93, 82, 94, 81, 95, 80, 92, 83, 91, 84, 90, 85, 89, 86, 88, 87],
       [109, 98, 110, 97, 111, 96, 108, 99, 107, 100, 106, 101, 105, 102,
        104, 103],
       [125, 114, 126, 113, 127, 112, 124, 115, 123, 116, 122, 117, 121,
        118, 120, 119]], dtype=object)

## Set probe configuration
Lets create probe configuration for a 128-channel probe

In [3]:
%matplotlib widget
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

#--- cambridge probe -----------
for i in range(8):
    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=16.5,
        ypitch=15,
        y_shift_per_column=[0,-7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    # shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = Probe(shanks)

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)

prbgrp.filename = sess.filePrefix.with_suffix(".probegroup.npy")
prbgrp.save()
plot_probe(prbgrp)



data saved


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'Probe 128ch'}>

## Create epochs for your experimental paradigm
A typical experiment involves multiple epochs such pre sleep, running on track and then another sleep epoch.

In [ ]:
sess.paradigm.to_dataframe()

In [ ]:
import pandas as pd
from neuropy.core import Epoch

epochs = pd.DataFrame(
    {
        "start": [0, 10000, 20000],
        "stop": [9000, 13600, 30000],
        "label": ["pre", "maze", "post"],
    }
)

a = Epoch(epochs=epochs)
a.filename = sess.filePrefix.with_suffix(".paradigm.npy")
# a.save()


# Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

## Artifacts epochs
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
signal = sess.eegfile.get_signal([1])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=7)
artifact_epochs.filename = sess.filePrefix.with_suffix(".artifact.npy")
pbe.filename = sess.filePrefix.with_suffix(".pbe")
plotting.plot_artifact_epochs(artifact_epochs, signal)


## Ripple epochs
To detect ripples one also needs probegroup.

In [ ]:
from neuropy.analyses import oscillations
signal = sess.eegfile.get_signal()
ripple_epochs =oscillations.detect_ripple_epochs(signal, sess.probegroup)
pbe.filename = sess.filePrefix.with_suffix(".pbe")
ripple_epochs.save()

In [ ]:
signal = sess.eegfile.get_signal(channel_id=[1, 2, 3, 4], t_start=1, t_stop=1.2)
plotting.plot_signal_traces(signal)

# Importing spiketrains from Phy

In [6]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path(
    "/data/Clustering/SleepDeprivation/RatK/Day1/spykcirc/RatK_Day1_2019-08-06_03-44-01/RatK_Day1_2019-08-06_03-44-01.GUI"
)
chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains =phy_data.spiketrains
peak_chans = phy_data.peak_channels
waveforms = phy_data.waveforms
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms,dtype='object'),
    shank_ids=np.array(shank_id).astype(int)
)

neurons.filename = sess.filePrefix.with_suffix('.neurons')
neurons.save()

data saved


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from neuropy.plotting import plot_raster


plt.plot(phy_data.peak_waveforms[0])
plot_raster(neurons,color='jet',add_vert_jitter=True)

# BinnedSpiketrain and Mua objects using Neurons

In [8]:
mua =sess.neurons.get_mua()
mua.filename = sess.filePrefix.with_suffix(".mua.npy")
mua.save()   


data saved


In [10]:
%matplotlib widget
from neuropy import plotting
smth_mua = sess.mua.get_smoothed(sigma=0.02)
# plotting.plot_mua(smth_mua)

In [11]:
from neuropy.analyses import detect_pbe_epochs

pbe = detect_pbe_epochs(smth_mua)
pbe.filename = sess.filePrefix.with_suffix('.pbe')
pbe.save()

/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


data saved
